In [1]:
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [2]:
response

<OpenAIObject chat.completion id=chatcmpl-7gNysffF7KtI8Un6AMRWJO165vUVV at 0x1107b8090> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played in Arlington, Texas at Globe Life Field.",
        "role": "assistant"
      }
    }
  ],
  "created": 1690336314,
  "id": "chatcmpl-7gNysffF7KtI8Un6AMRWJO165vUVV",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 53,
    "total_tokens": 70
  }
}

In [3]:
from weave.monitoring import monitor
m = monitor("completions_3", project_name="open_ai_monitor_example", entity_name="timssweeney")
monitored_create = m(openai.ChatCompletion.create)

In [5]:
record = monitored_create(model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ])

In [6]:
record.get()

<OpenAIObject chat.completion id=chatcmpl-7gO0EZx90LM82JsV8awlveCsxWLP9 at 0x105926bb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The World Series in 2020 was played in Arlington, Texas at the Globe Life Field, which was the home stadium of the Texas Rangers.",
        "role": "assistant"
      }
    }
  ],
  "created": 1690336398,
  "id": "chatcmpl-7gO0EZx90LM82JsV8awlveCsxWLP9",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 53,
    "total_tokens": 82
  }
}

In [7]:
monitored_create.rows()

In [ ]:
import weave
from weave import weave_internal as internal
from weave import weave_types as types

panels = weave.panels
ops = weave.ops


def board(
    input_node,
) -> panels.Group:
    control_items = [
        panels.GroupPanel(input_node, id="data"),
    ]
    data_var = internal.make_var_node(input_node.type, "data")

    clean_data = data_var.map(
        lambda row: ops.dict_(
            id=row["output"]["id"],
            object=row["output"]["object"],
            model=row["output"]["model"],
            messages=row["inputs"]["kwargs"]["messages"],
            usage=row["output"]["usage"],
            completion=row["output"]["choices"][0]["message"],
            finish_reason=row["output"]["choices"][0]["finish_reason"],
            timestamp=row["timestamp"],
        )
    )
    control_items.append(panels.GroupPanel(clean_data, id="clean_data"))
    clean_data_var = internal.make_var_node(clean_data.type, "clean_data")

    # Create a table from the query
    table = panels.Table(clean_data_var)
    table_state = table.config.tableState
    table_state.add_column(lambda row: row["model"], "Model")
    table_state.add_column(lambda row: row["messages"][-1]["content"], "Message")
    table_state.add_column(lambda row: row["completion"]["content"], "Completion")
    table_state.add_column(lambda row: row["usage"]["prompt_tokens"], "Prompt Tokens")
    table_state.add_column(
        lambda row: row["usage"]["completion_tokens"], "Completion Tokens"
    )
    table_state.add_column(lambda row: row["usage"]["total_tokens"], "Total Tokens")
    table_state.add_column(lambda row: row["timestamp"], "Timestamp")
    table_panel = panels.BoardPanel(table, id="table")


    board_panels = [
        table_panel,
    ]
    return panels.Board(vars=control_items, panels=board_panels)


board(monitored_create.rows())


In [ ]:
import weave
from weave import weave_internal as internal
from weave import weave_types as types

panels = weave.panels
ops = weave.ops


def board(
    input_node,
) -> panels.Group:
    # Define your VarBar variables here
    control_items = [
        #         panels.GroupPanel(internal.const("Example Custom Board"), id="title"),
        panels.GroupPanel(input_node, id="data"),
        panels.GroupPanel(
            lambda data: ops.make_list(
                a=data["timestamp"].min(), b=data["timestamp"].max()
            ),
            id="data_range",
            hidden=True,
        ),
        panels.GroupPanel(None, id="zoom_range", hidden=True),
        panels.GroupPanel(
            lambda zoom_range, data_range: zoom_range.coalesce(data_range),
            id="bin_range",
            hidden=True,
        ),
        panels.GroupPanel(
            lambda data, zoom_range: panels.DateRange(
                zoom_range, domain=data["timestamp"]
            ),
            id="date_picker",
        ),
    ]
    data_var = internal.make_var_node(input_node.type, "data")

    clean_data = data_var.map(
        lambda row: ops.dict_(
            id=row["output"]["id"],
            object=row["output"]["object"],
            model=row["output"]["model"],
            messages=row["inputs"]["kwargs"]["messages"],
            usage=row["output"]["usage"],
            completion=row["output"]["choices"][0]["message"],
            finish_reason=row["output"]["choices"][0]["finish_reason"],
            timestamp=row["timestamp"],
        )
    )
    control_items.append(panels.GroupPanel(clean_data, id="clean_data"))

    #     title_var = internal.make_var_node(input_node.type, "title")
    clean_data_var = internal.make_var_node(clean_data.type, "clean_data")

    # Create a table from the query
    table = panels.Table(clean_data_var)
    table_state = table.config.tableState

    def messages_col(row):
        print("messages_col", row.type)

        def message_pros(inner_row):
            print("message_pros", inner_row.type)
            return inner_row["role"]

        #         return row['messages'].map(message_pros)
        return row["messages"][-1]["content"]

    # Add columns, set sorting, and specify panels for columns
    table_state.add_column(lambda row: row["model"], "Model")
    table_state.add_column(lambda row: row["messages"][-1]["content"], "Message")
    table_state.add_column(lambda row: row["completion"]["content"], "Completion")
    table_state.add_column(lambda row: row["usage"]["prompt_tokens"], "Prompt Tokens")
    table_state.add_column(
        lambda row: row["usage"]["completion_tokens"], "Completion Tokens"
    )
    table_state.add_column(lambda row: row["usage"]["total_tokens"], "Total Tokens")
    table_state.add_column(lambda row: row["timestamp"], "Timestamp")
    table_panel = panels.BoardPanel(table, id="table")

    # Define your panels here
    board_panels = [
        panels.BoardPanel(
            panels.Plot(
                clean_data_var,
                x=lambda item: item["timestamp"].bin(
                    ops.timestamp_bins_nice(
                        internal.make_var_node(
                            types.List(types.Timestamp()), "bin_range"
                        ),
                        100,
                    )
                ),
                y=lambda preds: preds["usage"]["total_tokens"],
            ),
            id="total_token_usage",
        ),
        table_panel,
    ]
    return panels.Board(vars=control_items, panels=board_panels)


board(monitored_create.rows())
